In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras  import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [ ]:
procecssed_data_dir = r"D://ok\data_final//data"
# test_data_dir = "/home/divyansh/Desktop/ee/gray_ok/Grayscale Face images/test"
# validation_data_dir = "/home/divyansh/Desktop/ee/gray_ok/Grayscale Face images/validation"

In [9]:
# Define ImageDataGenerator with moderate augmentations
train_datagen = ImageDataGenerator(
    rescale=1./255,       # Normalize pixel values
    rotation_range=20,    # Rotate up to 20 degrees
    width_shift_range=0.1, # Shift image horizontally
    height_shift_range=0.1, # Shift image vertically
    shear_range=0.1,      # Apply shearing
    zoom_range=0.15,       # Random zoom
    horizontal_flip=True,  # Flip images
    fill_mode='nearest',   # Fill missing pixels
    validation_split=0.5   # 80% train, 20% validation
)

# Load training images
train_generator = train_datagen.flow_from_directory(
    procecssed_data_dir,# Path to dataset
    target_size=(323, 228),  # Resize images for CNN
    batch_size=32,     # Batch size
    class_mode='categorical',  # Multi-class classification
    subset='training'
)

# Load validation images (No Augmentation)
valid_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
valid_generator = valid_datagen.flow_from_directory(
    procecssed_data_dir,
    target_size=(323,228),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 23894 images belonging to 6 classes.
Found 9556 images belonging to 6 classes.


In [10]:
# Image size
IMG_WIDTH = 228
IMG_HEIGHT = 323
BATCH_SIZE = 32
NUM_CLASSES = 6
# Load the MobileNetV2 model pre-trained on ImageNet without the top layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

# Freeze the base model layers to retain pre-trained weights
base_model.trainable = False

# Add custom layers on top of MobileNetV2
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Pooling layer to reduce dimensionality
x = Dropout(0.5)(x)  # Add dropout for regularization
x = Dense(1024, activation='relu')(x)  # Fully connected layer
predictions = Dense(NUM_CLASSES, activation='softmax')(x)  # Output layer for classification

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# # Train the model



C:\Users\Mukesh\AppData\Local\Temp\ipykernel_16772\3028253323.py:7: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 323, 228,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 162, 114,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 162, 114,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 162, 114,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 162, 114,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 162, 114,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 162, 114,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 162, 114,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 162, 114,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 162, 114,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 162, 114,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 162, 114,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 163, 115,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 81, 57,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 81, 57,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 81, 57,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 81, 57,    │      2,304 │ block_1_depthwis

 Total params: 3,575,878 (13.64 MB)

 Trainable params: 1,317,894 (5.03 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [11]:
model.fit(
    train_generator,
    epochs=1 0,  # Adjust the number of epochs as necessary
    validation_data=valid_generator,
    verbose=1
)

Epoch 1/10
747/747 ━━━━━━━━━━━━━━━━━━━━ 1065s 1s/step - accuracy: 0.3794 - loss: 1.5585 - val_accuracy: 0.2479 - val_loss: 1.8690
Epoch 2/10
747/747 ━━━━━━━━━━━━━━━━━━━━ 885s 1s/step - accuracy: 0.6165 - loss: 1.0528 - val_accuracy: 0.2379 - val_loss: 1.9366
Epoch 3/10
747/747 ━━━━━━━━━━━━━━━━━━━━ 1644s 2s/step - accuracy: 0.6551 - loss: 0.9433 - val_accuracy: 0.2063 - val_loss: 2.0015
Epoch 4/10
747/747 ━━━━━━━━━━━━━━━━━━━━ 858s 1s/step - accuracy: 0.6921 - loss: 0.8615 - val_accuracy: 0.2351 - val_loss: 2.0267
Epoch 5/10
747/747 ━━━━━━━━━━━━━━━━━━━━ 946s 1s/step - accuracy: 0.7131 - loss: 0.8072 - val_accuracy: 0.2725 - val_loss: 1.9817
Epoch 6/10
747/747 ━━━━━━━━━━━━━━━━━━━━ 648s 868ms/step - accuracy: 0.7265 - loss: 0.7659 - val_accuracy: 0.2576 - val_loss: 2.1755
Epoch 7/10
747/747 ━━━━━━━━━━━━━━━━━━━━ 655s 877ms/step - accuracy: 0.7544 - loss: 0.7059 - val_accuracy: 0.2405 - val_loss: 2.2175
Epoch 8/10
747/747 ━━━━━━━━━━━━━━━━━━━━ 707s 947ms/step - accuracy: 0.7596 - loss: 0.6687

In [13]:
model.save("trained.h5")

In [ ]:
from transformers import pipeline

def create_summarizer():
    """
    Creates and returns a summarization function.
 
    Returns:
        function: A function that takes text, max_length, and min_length as input and returns a summary.
    """
    summarizer = pipeline("summarization")

    def summarize(text, max_length=150, min_length=10):
        """
        Summarizes the given text.

        Args:
            text (str): The text to summarize.
            max_length (int): The maximum length of the summary.
            min_length (int): The minimum length of the summary.

        Returns:
            str: The summarized text, or an error message.
        """
        try:
            summary = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)[0]['summary_text']
            return summary
        except Exception as e:
            return f"Error during summarization: {e}"

    return summarize

# Example usage:
summarize_function = create_summarizer()


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
# %pip install textstat

from transformers import pipeline
from textblob import TextBlob
import spacy
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import textstat

# Download necessary NLTK resources
nltk.download('vader_lexicon', quiet=True)
nlp = spacy.load("en_core_web_sm")

def create_enhanced_analyzer():
    """
    Creates and returns a function that performs summarization, sentiment analysis,
    polarity, subjectivity, and other text analysis.
    """
    summarizer = pipeline("summarization")
    sentiment_analyzer = pipeline("sentiment-analysis")
    sia = SentimentIntensityAnalyzer()

    def analyze_text(text, max_length=100, min_length=10):
        """
        Analyzes the given text, providing summarization, sentiment, polarity, subjectivity, and more.
        """
        results = {}

        try:
            summary = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)[0]['summary_text']
            results['summary'] = summary
        except Exception as e:
            results['summary'] = f"Error during summarization: {e}"

        try:
            sentiment_result = sentiment_analyzer(text)[0]
            results['sentiment'] = sentiment_result['label']
            results['sentiment_score'] = sentiment_result['score']
        except Exception as e:
            results['sentiment'] = f"Error during sentiment analysis: {e}"
            results['sentiment_score'] = None

        blob = TextBlob(text)
        results['polarity'] = blob.sentiment.polarity
        results['subjectivity'] = blob.sentiment.subjectivity
        results['words'] = len(blob.words)
        results['sentences'] = len(blob.sentences)
        # results['noun_phrases'] = blob.noun_phrases
        # results['word_counts'] = blob.word_counts
        # results['sentiment_detailed'] = [sentence.sentiment for sentence in blob.sentences]

        # Additional Extractions:

        # Named Entity Recognition (NER)
        doc = nlp(text)
        # results['named_entities'] = [(ent.text, ent.label_) for ent in doc.ents]

        # POS Tagging
        # tokens = nltk.word_tokenize(text)
        # results['pos_tags'] = nltk.pos_tag(tokens)

        # Sentiment Intensity (VADER)
        results['vader_sentiment'] = sia.polarity_scores(text)

        # Readability Score
        results['flesch_kincaid_grade'] = textstat.flesch_kincaid_grade(text)

        return results

    return analyze_text

# Example usage:
analyze_function = create_enhanced_analyzer()

text_to_analyze = """
Happiness is a fundamental goal that many people strive to achieve throughout their lives. It is a state of well-being characterized by positive or pleasant emotions ranging from contentment to intense joy, and it is often the ultimate aim of the decisions people make daily.
 However, the concept of happiness is subjective and varies from person to person. Some believe that happiness can be found in material wealth, while others find it in relationships, professional success, or personal fulfillment.

To truly understand happiness, it is essential to recognize that it is not a constant state but rather a journey that requires effort and self-awareness. Aristotle once said, "Happiness is the meaning and the purpose of life, the whole aim and end of human existence," highlighting the importance of happiness as a life goal.
 Achieving happiness involves understanding what it means to be happy and how to create it in one's life. This process is unique to each individual, as everyone has their own specific characteristics and defining traits that influence their path to happiness.

One key aspect of happiness is the realization that it is not solely dependent on external factors such as wealth or status. While money can provide comfort and security, it does not guarantee happiness. The rich are often burdened with stress, anxiety, and insecurity, whereas the poor, despite their lack of material possessions, can find joy in simpler pleasures and stronger social bonds.

Another important factor in achieving happiness is the development of positive relationships. Spending quality time with family and friends, expressing gratitude, and maintaining a positive outlook on life can significantly contribute to one's overall happiness.
 Positive thoughts and a positive mindset attract positive experiences, making it easier to sustain a state of happiness.

Moreover, happiness can be cultivated through habits that promote well-being. Practicing mindfulness, transforming intense emotions into calmness and contentment, and focusing on the present moment can help maintain a state of happiness.
 Additionally, it is crucial to let go of past mistakes and not worry excessively about the future, as dwelling on negative thoughts can hinder one's ability to experience joy.

In conclusion, happiness is a complex and multifaceted concept that requires personal effort and self-reflection. It is not about achieving perfection or external validation but rather about finding contentment and joy in one's life experiences and relationships. By understanding the true meaning of happiness and actively working towards it, individuals can lead a more fulfilling and joyful life.


"""

analysis_results = analyze_function(text_to_analyze)
print(analysis_results)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Device set to use cpu
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


{'summary': ' Happiness is a state of well-being characterized by positive emotions ranging from contentment to intense joy . Aristotle once said, "Happiness is the meaning and the purpose of life, the whole aim and end of human existence" Achieving happiness involves understanding what it means to be happy and how to create it .', 'sentiment': 'POSITIVE', 'sentiment_score': 0.9988752007484436, 'polarity': 0.3162419651056014, 'subjectivity': 0.4123393021120293, 'words': 413, 'sentences': 20, 'vader_sentiment': {'neg': 0.054, 'neu': 0.566, 'pos': 0.38, 'compound': 0.9996}, 'flesch_kincaid_grade': 13.6}


In [ ]:
question1 = "What has brought you to therapy?"
text1 = input("Enter your answer here")
print("\n")
summary1 = analyze_function(text1)

print("Summary 1:\n", summary1)

Your max_length is set to 150, but your input_length is only 127. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=63)




Summary 1:
 {'summary': ' I am writing to inform you that I am feeling quite unwell today and will not be able to attend our scheduled session . I sincerely apologize for any inconvenience this may cause . I understand the importance of our session and am committed to making sure we can continue our work effectively .', 'sentiment': 'POSITIVE', 'sentiment_score': 0.9941498041152954, 'polarity': 0.6, 'subjectivity': 0.6448412698412699, 'words': 111, 'sentences': 8, 'vader_sentiment': {'neg': 0.019, 'neu': 0.636, 'pos': 0.345, 'compound': 0.9882}, 'flesch_kincaid_grade': 7.5}


In [ ]:
question1 = "Are there any aspects of your daily life that you struggle with right now?"
text2 = input("Enter your answer here")
summary2 = analyze_function(text2, max_length=80, min_length=20) 
print("\nSummary 2:\n", summary2)


Summary 2:
 {'summary': ' Dizziness can also be a symptom of inner ear disorders, cardiovascular issues, neurological diseases, or even anxiety and stress . Postural orthostatic tachycardia syndrome (POTS) is a condition that is often missed and misdiagnosed .', 'sentiment': 'NEGATIVE', 'sentiment_score': 0.9965212345123291, 'polarity': -0.020833333333333332, 'subjectivity': 0.4861111111111111, 'words': 104, 'sentences': 5, 'vader_sentiment': {'neg': 0.094, 'neu': 0.754, 'pos': 0.152, 'compound': 0.6808}, 'flesch_kincaid_grade': 14.9}


In [ ]:
question2 = "Where do you go for support when you need it? Do you have a network of supportive relationships with others?"
text3 = input("Enter your answer here")
summary3 = analyze_function(text3, max_length=80, min_length=20)
print("\nSummary 3:\n", summary3)


Summary 3:
 {'summary': ' Having a supportive network of family members is crucial because they can offer a sense of belonging and social support, especially during tough times . My family helps me combat feelings of loneliness and fosters a . sense of connection and well-being .', 'sentiment': 'POSITIVE', 'sentiment_score': 0.9995335340499878, 'polarity': 0.18827160493827164, 'subjectivity': 0.6444444444444444, 'words': 86, 'sentences': 5, 'vader_sentiment': {'neg': 0.07, 'neu': 0.654, 'pos': 0.276, 'compound': 0.9554}, 'flesch_kincaid_grade': 10.0}


In [ ]:
question3 ="How are your relationships with your family? How often are you in touch with them?"
text4 = input("Enter your answer here")
summary4 = analyze_function(text4)
print("\nSummary 4:\n", summary4)


Summary 4:
 {'summary': " Family relationships play a central role in shaping an individual's well-being, and staying connected with family members is crucial for maintaining strong bonds . I value my relationships with my family and make an effort to stay in touch regularly . I often communicate with them through various means such as phone calls, emails, and video calls .", 'sentiment': 'POSITIVE', 'sentiment_score': 0.9990866184234619, 'polarity': 0.11349206349206346, 'subjectivity': 0.527960927960928, 'words': 150, 'sentences': 7, 'vader_sentiment': {'neg': 0.022, 'neu': 0.777, 'pos': 0.2, 'compound': 0.9761}, 'flesch_kincaid_grade': 12.6}


In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load your trained model
model = tf.keras.models.load_model("best_model.keras")  # Replace with your model path

# Define emotion labels (adjust to your model's output)
emotion_labels = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]

# Function to preprocess the frame
def preprocess_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (48, 48))
    normalized = resized / 255.0
    reshaped = np.reshape(normalized, (1, 48, 48, 1))
    return reshaped

# Open the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    processed_frame = preprocess_frame(frame)

    predictions = model.predict(processed_frame)
    emotion_index = np.argmax(predictions)
    emotion = emotion_labels[emotion_index]
    confidence = predictions[0][emotion_index]

    text = f"{emotion} ({confidence:.2f})"
    cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Emotion Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

ValueError: File not found: filepath=best_model.keras. Please ensure the file is an accessible `.keras` zip file.